## Step 1: Setup and Installation

Install required packages and check GPU availability.

In [ ]:
# Install CuPy for GPU computing (choose based on CUDA version)
# For Colab (usually CUDA 11.x or 12.x):
!pip install cupy-cuda11x -q

# If above fails, try:
# !pip install cupy-cuda12x -q

print("✓ Installation complete!")

In [ ]:
import numpy as np
import cupy as cp
from scipy import sparse as sp
import time
import json
import csv
from pathlib import Path
import os

print("GPU Available:", cp.cuda.is_available())
if cp.cuda.is_available():
    device = cp.cuda.Device()
    print(f"GPU Device: {device}")
    print(f"GPU Memory: {device.mem_info[1] / 1e9:.2f} GB")
else:
    print("⚠️ No GPU detected! Make sure Runtime → Change runtime type → GPU")

## Step 2: Upload Data Files

Upload your sparse matrix CSV files (matrix_a.csv, matrix_b.csv) from the `data/input` folder.

**Format**: Each line should be: `row,col,value` (1-based indexing)

In [ ]:
from google.colab import files

print("Please upload matrix_a.csv and matrix_b.csv")
uploaded = files.upload()

print("\n✓ Files uploaded:")
for filename in uploaded.keys():
    size_mb = len(uploaded[filename]) / 1e6
    print(f"  - {filename}: {size_mb:.2f} MB")

## Step 3: Define Block Multiplication Class

Core implementation of block-based multiplication.

In [ ]:
class BlockMatrixMultiplier:
    """
    Block-based matrix multiplication for large matrices on GPU.
    """
    
    def __init__(self, gpu_memory_gb=12, safety_factor=0.7):
        """
        Initialize block multiplier.
        
        Args:
            gpu_memory_gb: Available GPU memory in GB
            safety_factor: Use only 70% of memory to be safe
        """
        self.gpu_memory_bytes = int(gpu_memory_gb * 1e9 * safety_factor)
        self.gpu_available = cp.cuda.is_available()
        
        if self.gpu_available:
            device = cp.cuda.Device()
            actual_memory = device.mem_info[1] / 1e9
            print(f"✓ GPU Available: {device}")
            print(f"✓ Total GPU Memory: {actual_memory:.1f}GB")
            print(f"✓ Using: {gpu_memory_gb * safety_factor:.1f}GB ({safety_factor*100:.0f}%)")
        else:
            raise RuntimeError("No GPU available!")
    
    def estimate_block_size(self, matrix_size, sparsity=0.99):
        """
        Estimate optimal block size based on available GPU memory.
        """
        # Conservative estimate
        bytes_per_element = 8  # float32 + overhead
        max_elements = self.gpu_memory_bytes // bytes_per_element
        
        # block_size × matrix_size × 2 (A block + B block)
        block_size = int(max_elements / (2 * matrix_size))
        
        # Clamp to reasonable range
        block_size = min(block_size, matrix_size, 5000)
        block_size = max(block_size, 100)
        
        num_blocks = int(np.ceil(matrix_size / block_size))
        memory_per_block_gb = block_size * matrix_size * 4 * 2 / 1e9
        
        print(f"\n{'='*60}")
        print(f"Memory Estimation:")
        print(f"{'='*60}")
        print(f"Matrix size: {matrix_size:,} × {matrix_size:,}")
        print(f"Block size: {block_size:,} × {block_size:,}")
        print(f"Number of blocks: {num_blocks} × {num_blocks} = {num_blocks**2}")
        print(f"Memory per block: ~{memory_per_block_gb:.2f}GB")
        print(f"{'='*60}\n")
        
        return block_size
    
    def load_sparse_csv(self, filepath, matrix_size):
        """
        Load sparse matrix from CSV file.
        """
        print(f"Loading: {filepath}...")
        
        rows, cols, vals = [], [], []
        
        with open(filepath, 'r') as f:
            reader = csv.reader(f)
            for parts in reader:
                if len(parts) == 3:
                    try:
                        # Convert 1-based to 0-based indexing
                        r = int(parts[0]) - 1
                        c = int(parts[1]) - 1
                        v = float(parts[2])
                        
                        if 0 <= r < matrix_size and 0 <= c < matrix_size:
                            rows.append(r)
                            cols.append(c)
                            vals.append(v)
                    except ValueError:
                        continue
        
        nnz = len(vals)
        sparsity = 100 * (1 - nnz / (matrix_size * matrix_size))
        
        print(f"  ✓ {nnz:,} non-zero entries")
        print(f"  ✓ {sparsity:.4f}% sparse\n")
        
        # Create CSR matrix
        sparse_mat = sp.csr_matrix((vals, (rows, cols)), 
                                   shape=(matrix_size, matrix_size))
        
        return sparse_mat
    
    def multiply_sparse_block(self, A_csr, B_csr, row_start, row_end, 
                               col_start, col_end):
        """
        Multiply a block of A with a block of B on GPU.
        """
        # Extract blocks
        A_block = A_csr[row_start:row_end, :].toarray()
        B_block = B_csr[:, col_start:col_end].toarray()
        
        # Transfer to GPU
        A_gpu = cp.asarray(A_block, dtype=cp.float32)
        B_gpu = cp.asarray(B_block, dtype=cp.float32)
        
        # Multiply on GPU
        C_gpu = cp.matmul(A_gpu, B_gpu)
        
        # Transfer back to CPU
        C_block = cp.asnumpy(C_gpu)
        
        # Clean up GPU memory
        del A_gpu, B_gpu, C_gpu
        cp.get_default_memory_pool().free_all_blocks()
        
        return C_block
    
    def multiply_blocks(self, A_csr, B_csr, block_size, output_dir="./blocks"):
        """
        Multiply two matrices using block algorithm.
        """
        matrix_size = A_csr.shape[0]
        
        # Create output directory
        os.makedirs(output_dir, exist_ok=True)
        
        # Calculate blocks
        num_blocks = int(np.ceil(matrix_size / block_size))
        total_blocks = num_blocks * num_blocks
        
        print(f"\n{'='*60}")
        print(f"Starting Block Multiplication")
        print(f"{'='*60}")
        print(f"Total blocks: {num_blocks} × {num_blocks} = {total_blocks}")
        print(f"Output: {output_dir}")
        print(f"{'='*60}\n")
        
        metadata = {
            'matrix_size': matrix_size,
            'block_size': block_size,
            'num_blocks': num_blocks,
            'blocks': []
        }
        
        total_time = 0
        block_count = 0
        
        # Process each block
        for i in range(num_blocks):
            row_start = i * block_size
            row_end = min(row_start + block_size, matrix_size)
            
            for j in range(num_blocks):
                col_start = j * block_size
                col_end = min(col_start + block_size, matrix_size)
                
                block_count += 1
                
                print(f"[{block_count}/{total_blocks}] Block ({i},{j}): "
                      f"rows [{row_start}:{row_end}], cols [{col_start}:{col_end}]", 
                      end=" ")
                
                # Multiply
                start = time.perf_counter()
                C_block = self.multiply_sparse_block(
                    A_csr, B_csr, row_start, row_end, col_start, col_end
                )
                elapsed = time.perf_counter() - start
                total_time += elapsed
                
                # Save
                filename = f"block_{i}_{j}.npy"
                np.save(os.path.join(output_dir, filename), C_block)
                
                nnz = np.count_nonzero(C_block)
                print(f"→ {nnz:,} nnz, {elapsed:.3f}s ✓")
                
                metadata['blocks'].append({
                    'i': i, 'j': j,
                    'row_start': row_start, 'row_end': row_end,
                    'col_start': col_start, 'col_end': col_end,
                    'filename': filename,
                    'time': elapsed
                })
        
        # Save metadata
        metadata_path = os.path.join(output_dir, 'metadata.json')
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=2)
        
        print(f"\n{'='*60}")
        print(f"✓ Complete! Total: {total_time:.2f}s, Avg: {total_time/total_blocks:.3f}s/block")
        print(f"✓ Metadata: {metadata_path}")
        print(f"{'='*60}\n")
        
        return metadata_path
    
    def reconstruct_result(self, metadata_path):
        """
        Reconstruct full result matrix from blocks.
        """
        print("\nReconstructing result matrix...")
        
        with open(metadata_path, 'r') as f:
            metadata = json.load(f)
        
        matrix_size = metadata['matrix_size']
        output_dir = os.path.dirname(metadata_path)
        
        # Collect sparse entries
        rows, cols, vals = [], [], []
        
        for block_info in metadata['blocks']:
            block_path = os.path.join(output_dir, block_info['filename'])
            C_block = np.load(block_path)
            
            row_start = block_info['row_start']
            col_start = block_info['col_start']
            
            # Extract non-zeros
            block_rows, block_cols = np.nonzero(C_block)
            for r, c in zip(block_rows, block_cols):
                rows.append(row_start + r)
                cols.append(col_start + c)
                vals.append(C_block[r, c])
        
        result_csr = sp.csr_matrix((vals, (rows, cols)), 
                                   shape=(matrix_size, matrix_size))
        
        nnz = result_csr.nnz
        sparsity = 100 * (1 - nnz / matrix_size**2)
        
        print(f"✓ Result: {nnz:,} non-zeros, {sparsity:.4f}% sparse\n")
        
        return result_csr

print("✓ BlockMatrixMultiplier class defined!")

## Step 4: Configure and Run Block Multiplication

Set parameters and execute the multiplication.

In [ ]:
# Configuration
MATRIX_SIZE = 50000  # Adjust based on your data
GPU_MEMORY_GB = 12   # Google Colab typically has 12-15GB

# File paths
MATRIX_A_FILE = "matrix_a.csv"
MATRIX_B_FILE = "matrix_b.csv"
OUTPUT_DIR = "./block_results"

print(f"Configuration:")
print(f"  Matrix size: {MATRIX_SIZE:,} × {MATRIX_SIZE:,}")
print(f"  GPU memory: {GPU_MEMORY_GB}GB")
print(f"  Output dir: {OUTPUT_DIR}")

In [ ]:
# Initialize multiplier
multiplier = BlockMatrixMultiplier(gpu_memory_gb=GPU_MEMORY_GB)

# Estimate optimal block size
block_size = multiplier.estimate_block_size(MATRIX_SIZE, sparsity=0.99)

In [ ]:
# Load matrices
print("Loading matrices...\n")
A_csr = multiplier.load_sparse_csv(MATRIX_A_FILE, MATRIX_SIZE)
B_csr = multiplier.load_sparse_csv(MATRIX_B_FILE, MATRIX_SIZE)

In [ ]:
# Multiply using blocks (this will take some time!)
metadata_path = multiplier.multiply_blocks(A_csr, B_csr, block_size, OUTPUT_DIR)

## Step 5: Reconstruct and Save Result

Combine all blocks into the final result matrix.

In [ ]:
# Reconstruct result
result_csr = multiplier.reconstruct_result(metadata_path)

# Save result
result_file = os.path.join(OUTPUT_DIR, "result_matrix.npz")
sp.save_npz(result_file, result_csr)

print(f"\n✓ Final result saved: {result_file}")
print(f"  Shape: {result_csr.shape}")
print(f"  Non-zeros: {result_csr.nnz:,}")
print(f"  File size: {os.path.getsize(result_file) / 1e6:.2f} MB")

## Step 6: Download Results

Download the result files to your local machine.

In [ ]:
from google.colab import files

# Download result matrix
files.download(result_file)

# Download metadata (for reference)
files.download(metadata_path)

print("\n✓ Files ready for download!")

## Optional: Verification

Compare a small sample with CPU computation to verify correctness.

In [ ]:
# Verify with small sample
print("Verification: Computing small sample on CPU...\n")

# Take first 100x100 block
sample_size = 100
A_sample = A_csr[:sample_size, :sample_size]
B_sample = B_csr[:sample_size, :sample_size]
result_sample = result_csr[:sample_size, :sample_size]

# Compute on CPU
expected = A_sample @ B_sample

# Compare
diff = np.abs(expected - result_sample).max()

print(f"Maximum difference: {diff}")
if diff < 1e-4:
    print("✓ Verification PASSED! Results match.")
else:
    print("⚠️ Warning: Results differ slightly (may be due to floating point precision)")

## Summary

### What We Accomplished:

✅ **Loaded large sparse matrices** from CSV files  
✅ **Estimated optimal block size** based on GPU memory  
✅ **Multiplied matrices in chunks** that fit in GPU memory  
✅ **Saved intermediate results** to disk  
✅ **Reconstructed final result** from blocks  

### Key Advantages:

- 🚀 **Scalable**: Can handle matrices of ANY size
- 💾 **Memory-efficient**: Only loads small chunks into GPU
- ⚡ **Fast**: Uses GPU for actual computation
- 🔄 **Flexible**: Adjusts block size based on available memory

### Comparison:

| Method | Memory | Speed | Scalability |
|--------|--------|-------|-------------|
| **Dense CPU** | 80GB+ | Slow | ❌ Limited |
| **Dense GPU (naive)** | 80GB+ | **CRASHES** | ❌ Fails |
| **Block GPU (this)** | 12GB | Fast | ✅ **Unlimited** |

---

### Next Steps:

1. Adjust `MATRIX_SIZE` if your matrices are different
2. Tune `block_size` for your specific GPU memory
3. Save result in different formats (CSV, dense, etc.)
4. Compare performance with CPU sparse multiplication